In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/benjaminrike1/social_score_csr_reports.git

Cloning into 'social_score_csr_reports'...
remote: Enumerating objects: 259, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 259 (delta 14), reused 43 (delta 13), pack-reused 215
Receiving objects: 100% (259/259), 786.58 MiB | 8.19 MiB/s, done.
Resolving deltas: 100% (83/83), done.
Checking out files: 100% (79/79), done.


In [4]:
pip install yfinance

     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 6.4 MB 11.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [5]:
import yfinance as yf
import pandas as pd

In [29]:
data_path = 'social_score_csr_reports/'
social_scores = pd.read_csv(data_path+'social_scores.csv', index_col=0)

In [25]:
# reading in company-ticker mapping
df_financial = pd.read_csv(data_path+'Tickers.csv', sep=';')
df_financial = df_financial[(df_financial['Ticker'])!='0'].reset_index(drop=True)
df_financial.head()

,Company,Ticker
0,ABNAMRO,ABN.AS
1,Amazon,AMZN
2,asr,ASRNL.AS
3,bam,BAMNB.AS
4,CocaCola,KO


In [31]:
# merging financial and social_scores df
merged = df_financial.merge(social_scores, on='Company')

In [33]:
merged.head(5)

,Company,Ticker,scores
0,ABNAMRO,ABN.AS,0.253972
1,Amazon,AMZN,0.322418
2,asr,ASRNL.AS,0.280407
3,bam,BAMNB.AS,0.381486
4,CocaCola,KO,0.317769


In [34]:
# downloading comapny info from Yahoo
information = df_financial['Ticker'].apply(lambda x: yf.Ticker(x).info)
information

0     {'zip': '1082 PP', 'sector': 'Financial Servic...
1     {'zip': '98109-5210', 'sector': 'Consumer Cycl...
2     {'zip': '3584 BA', 'sector': 'Financial Servic...
3     {'zip': '3981 AZ', 'sector': 'Industrials', 'f...
4     {'zip': '30313', 'sector': 'Consumer Defensive...
5     {'zip': 'NW1 3AX', 'sector': 'Consumer Cyclica...
6     {'zip': '101 23', 'sector': 'Consumer Defensiv...
7     {'zip': '94025', 'sector': 'Communication Serv...
8     {'zip': '6340', 'sector': 'Industrials', 'full...
9     {'zip': '94043', 'sector': 'Communication Serv...
10    {'zip': '1017 ZD', 'sector': 'Consumer Defensi...
11    {'zip': '40589', 'sector': 'Consumer Defensive...
12    {'zip': '94304', 'sector': 'Technology', 'full...
13    {'zip': '1102 CT', 'sector': 'Financial Servic...
14    {'zip': '95054-1549', 'sector': 'Technology', ...
15    {'zip': '3072 AP', 'sector': 'Communication Se...
16    {'zip': '98052-6399', 'sector': 'Technology', ...
17    {'zip': '2595 AS', 'sector': 'Financial Se

In [35]:
# extracting sector, exchange and currency as features
df_financial['sector'] = [x.get('sector') for x in information]
df_financial['exchange'] = [x.get('exchange') for x in information]
df_financial['currency'] = [x.get('currency') for x in information]

In [36]:
# extracting adujsted price for two dates
df_financial['adjusted_10.05.2020'] = df_financial['Ticker'].apply(lambda x: yf.download(x, '2020-05-11',progress=False)['Adj Close'].values[0])
df_financial['adjusted_10.05.2022'] = df_financial['Ticker'].apply(lambda x: yf.download(x, '2022-05-11',progress=False)['Adj Close'].values[0])

In [37]:
# adding exchange ticker to extract exchange prices
Ticker_exchange = {'NYQ': '^NYA', 'AMS': '^AEX', 'NMS': '^IXIC', 'LSE':'^FTSE' ,
 'STO':'^OMX', 'PNK':'^AEX','EBS':'^SSMI', 'GER':'^GDAXI', 'CPH':'^OMXC20'}
df_financial['Exchange_ticker'] = df_financial['exchange'].map(Ticker_exchange)
df_financial['Exchange_2020.05.10'] = df_financial['Exchange_ticker'].apply(lambda x: yf.download(x, '2020-05-10',progress=False).Close.values[0])
df_financial['Exchange_2022.05.10'] = df_financial['Exchange_ticker'].apply(lambda x: yf.download(x, '2022-05-10',progress=False).Close.values[0])

In [38]:
#Using dictionary to map currency to the Yahoo searchword for exchange rate

Valuta = {'EUR': 'EURUSD=x', 'DKK': 'DKKUSD=x', 'SEK': 'SEKUSD=x', 'CHF':'CHFUSD=x' ,
 'GBp':'GBPUSD=x', 'USD': 'USD=x'}
df_financial['valuta_ticker'] = df_financial['currency'].map(Valuta)

# Correcting for exchange rates for stock and stock exchange prices
df_financial['Exchange_rate_2020'] = df_financial['valuta_ticker'].apply(lambda x: yf.download(x, '2020-05-10', progress=False).Close[0])
df_financial['Exchange_rate_2022'] = df_financial['valuta_ticker'].apply(lambda x: yf.download(x, '2022-05-10', progress=False).Close[0])
df_financial['USD_adjusted_2020.10.05'] = df_financial['Exchange_rate_2020']*df_financial['adjusted_10.05.2020']
df_financial['USD_adjusted_2022.10.05'] = df_financial['Exchange_rate_2022']*df_financial['adjusted_10.05.2022']
df_financial['USD_Exchange_2020.10.05'] = df_financial['Exchange_rate_2020']*df_financial['Exchange_2020.05.10']
df_financial['USD_Exchange_2022.10.05'] = df_financial['Exchange_rate_2022']*df_financial['Exchange_2022.05.10']
df_financial.head()

,Company,Ticker,sector,exchange,currency,adjusted_10.05.2020,adjusted_10.05.2022,Exchange_ticker,Exchange_2020.05.10,Exchange_2022.05.10,valuta_ticker,Exchange_rate_2020,Exchange_rate_2022,USD_adjusted_2020.10.05,USD_adjusted_2022.10.05,USD_Exchange_2020.10.05,USD_Exchange_2022.10.05
0,ABNAMRO,ABN.AS,Financial Services,AMS,EUR,6.283122,11.105000,^AEX,520.330017,669.739990,EURUSD=x,1.083952,1.05658,6.810603,11.733319,564.012799,707.633764
1,Amazon,AMZN,Consumer Cyclical,NMS,USD,2409.000000,2107.439941,^IXIC,9192.339844,11737.669922,USD=x,1.000000,1.00000,2409.000000,2107.439941,9192.339844,11737.669922
2,asr,ASRNL.AS,Financial Services,AMS,EUR,19.876907,42.849998,^AEX,520.330017,669.739990,EURUSD=x,1.083952,1.05658,21.545615,45.274444,564.012799,707.633764
3,bam,BAMNB.AS,Industrials,AMS,EUR,1.478000,2.402000,^AEX,520.330017,669.739990,EURUSD=x,1.083952,1.05658,1.602081,2.537905,564.012799,707.633764
4,CocaCola,KO,Consumer Defensive,NYQ,USD,42.734631,64.309998,^NYA,11281.370117,15025.740234,USD=x,1.000000,1.00000,42.734631,64.309998,11281.370117,15025.740234


In [39]:
df_financial['return'] = (df_financial['USD_adjusted_2022.10.05']-df_financial['USD_adjusted_2020.10.05'])/(df_financial['USD_adjusted_2020.10.05'])
df_financial['exchange_return'] = (df_financial['USD_Exchange_2022.10.05']-df_financial['USD_Exchange_2020.10.05'])/(df_financial['USD_Exchange_2020.10.05'])

In [40]:
df_financial.head()

,Company,Ticker,sector,exchange,currency,adjusted_10.05.2020,adjusted_10.05.2022,Exchange_ticker,Exchange_2020.05.10,Exchange_2022.05.10,valuta_ticker,Exchange_rate_2020,Exchange_rate_2022,USD_adjusted_2020.10.05,USD_adjusted_2022.10.05,USD_Exchange_2020.10.05,USD_Exchange_2022.10.05,return,exchange_return
0,ABNAMRO,ABN.AS,Financial Services,AMS,EUR,6.283122,11.105000,^AEX,520.330017,669.739990,EURUSD=x,1.083952,1.05658,6.810603,11.733319,564.012799,707.633764,0.722802,0.254641
1,Amazon,AMZN,Consumer Cyclical,NMS,USD,2409.000000,2107.439941,^IXIC,9192.339844,11737.669922,USD=x,1.000000,1.00000,2409.000000,2107.439941,9192.339844,11737.669922,-0.125181,0.276897
2,asr,ASRNL.AS,Financial Services,AMS,EUR,19.876907,42.849998,^AEX,520.330017,669.739990,EURUSD=x,1.083952,1.05658,21.545615,45.274444,564.012799,707.633764,1.101330,0.254641
3,bam,BAMNB.AS,Industrials,AMS,EUR,1.478000,2.402000,^AEX,520.330017,669.739990,EURUSD=x,1.083952,1.05658,1.602081,2.537905,564.012799,707.633764,0.584130,0.254641
4,CocaCola,KO,Consumer Defensive,NYQ,USD,42.734631,64.309998,^NYA,11281.370117,15025.740234,USD=x,1.000000,1.00000,42.734631,64.309998,11281.370117,15025.740234,0.504868,0.331907


In [ ]:
# list of features that should be used
features = ['sector', 'exchange', 'adujusted_10.05']
label = '' # label that should be predicted

In [ ]:
# using statsmodels to complete proper extraction of data
import statsmodels.api as sm
